# はじめてのPython入門
## 第3章　Pythonでデータ分析をしてみよう！　ハンズオン
<div align="right">
タナカケンタ<br/>
https://mana.bi/
</div>

### 3.4　演習: Pythonで探索的データ解析

ここでは、Pythonと各種ライブラリを使って、探索的データ解析 (EDA; Exploratory Data Analysis) を行います。探索的データ解析とは、手元にあるデータを、さまざまな観点から集計、可視化して観察することで、データの特徴やデータどうしの関係性を理解しようとするアプローチです。

のちほど取り上げる、機械学習などを使った高度な分析を行う前に、分析の方向性や適用する手法を決めるために重要なアプローチです。

今回は、データ分析の練習・学習用によく使われる、[レンタル自転車の利用状況に関するデータセット](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset)を用います。このデータは、日別と1時間ごとの集計データの2つがあります。まずは、日別のデータを観察してみましょう。

はじめに、Googleドライブをマウントし、探索的データ解析に使用するライブラリを読み込みます。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

次に、データファイルを読み込みます。

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/zeropy/chap03/day.csv')
df.head()

ここから、データの観察をしていきましょう。まず、データフレーム全体の情報を確認します。Pandasでは、`info()` メソッドでデータフレーム全体の情報を出力できます。

In [ ]:
df.info()

どのような列があり、それぞれの列がどのような型であるかがわかります。なお、Pandasデータフレームにおける `object` 型は、主に文字列が格納されていることを意味します。

### 単純集計

まず、いくつかの列について、全行を合計した数量を求めます。データフレームの列の値を合計するには、2章でも紹介した `sum()` メソッドを使います。

ここでは、自転車の利用者数をあらわす、`casual` (一時利用者)、`registered` (定期利用者)、`cnt` (合計) について、合計を算出します。

In [ ]:
df[['casual', 'registered', 'cnt']].sum()

1年を通じての利用数は、一時利用者が約62万件、定期利用者が約267万件ということがわかり、このサービスは定期的に利用するユーザーが多いようです。

### グループ化集計 (クロス集計)

次に、いくつかの条件ごとに、利用者数の合計や平均が異なるのかを見てみましょう。まず、`yr` という変数は、0が2011年、1が2012年のデータであることをあらわしています。2011年と2012年で利用者数にどのくらいの差があるかを見てみましょう。

Pandasでは、`groupby()` メソッドで集計軸を指定し、続けて集計関数を指定できます。

In [ ]:
df[['yr', 'casual', 'registered', 'cnt']].groupby('yr').sum()

2011年から2012年にかけて、利用者が増加し、ビジネスが拡大していることがわかります。次に、`season` 列は季節をあらわす変数で、1: 冬、2: 春、3: 夏、4: 秋と表現されています。季節ごとの利用者数の合計と、それぞれの季節における1日の平均利用者数を見てみましょう。

In [ ]:
df[['season', 'casual', 'registered', 'cnt']].groupby('season').sum()

In [ ]:
df[['season', 'casual', 'registered', 'cnt']].groupby('season').mean()

季節別に見ると、春 (2) と夏 (3) の利用者が多い傾向にあります。

また、`casual` ユーザーは季節によって大きく増減するようです。秋冬に急に自転車に乗りたくなることは少ないと考えられます。一方、`registered` ユーザーはそれほど極端な増減はなく、定期利用者は年間を通じて自転車を利用する傾向にあるようです。

他にも、`workingday` (平日が1、土日祝日が0) や `weathersit` (晴れ / 多少の曇りが1、小雨・霧雨が2、雨・雪・雷が3、大雨・雷雨・ひょうなどが4) など、グループ化して集計することで、利用者の動向が読み取れる情報があるので、ぜひいろいろな観点での集計をしてみましょう。

### データの可視化

続いて、データを可視化 (グラフ化) してみましょう。Pythonでは、Matplotlibライブラリを使用するのが標準的です。PandasとMatplotlibは連携していて、Pandasデータフレームの `plot()` メソッドや `hist()` メソッドなどで、Matplotlibを呼び出してグラフィックスを作成できます。

ここでは、まず1日ごとの利用者数の合計 (`cnt`) を棒グラフにしてみましょう。

In [ ]:
fig,ax = plt.subplots(figsize=(16, 8))
plt.plot(pd.to_datetime(df['dteday'], format='%Y-%m'), df['cnt']);

全体として上昇傾向にありますが、突発的に利用者が少ない日があることがわかります。次に、気温 `temp` と利用者数の関係性について見てみましょう。

In [ ]:
fig,ax = plt.subplots(figsize=(16, 8))
plt.scatter(df['cnt'], df['temp']);

気温と利用者数の間には、正の相関関係がありそうです。相関係数も算出してみましょう。

In [ ]:
df.loc[:, ['cnt', 'temp']].corr()

### Sweetvizライブラリによる探索的データ解析

探索的データ解析を、PandasやMatplotlibの機能を使って行うことができますが、別の選択肢として、探索的データ解析に特化したライブラリを活用するアプローチがあります。

[Sweetviz](https://github.com/fbdesignpro/sweetviz)ライブラリは、マウス操作で多様な観点での観察ができます。Colabの中に表示することもできます。以下に例を示します。

In [ ]:
% pip install sweetviz

In [ ]:
import sweetviz as sv
report = sv.analyze(df)
report.show_notebook()

2値の変数を集計軸にして、他の変数を集計した結果も出力できます。ここでは、`yr` 列を集計軸としてみます。

In [ ]:
report2 = sv.compare_intra(df, df['yr'] == 0, ['2011年', '2012年'])
report2.show_notebook()